In [1]:
import os
import csv
from supabase import create_client, Client
import os
import datetime
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [18]:
COMPETITION_ID = 5
POOL_ID = 14
TODAY = datetime.date.today()
# TODAY = "2024-03-28"
CURRENT_ROUND = 5


In [14]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [15]:
round = CURRENT_ROUND
update_game_schedule(f"2024/game-schedule-round-{round}-2024-ncaa-tournament.csv")

{'game_date': '2024-03-30', 'team_2_id': 'illinois', 'team_1_id': 'connecticut', 'game_time': '18:09', 'round_num': '5', 'competition_id': '5'}
{'game_date': '2024-03-30', 'team_2_id': 'clemson', 'team_1_id': 'alabama', 'game_time': '20:49', 'round_num': '5', 'competition_id': '5'}
{'game_date': '2024-03-31', 'team_2_id': 'purdue', 'team_1_id': 'tennessee', 'game_time': '14:20', 'round_num': '5', 'competition_id': '5'}
{'game_date': '2024-03-31', 'team_2_id': 'duke', 'team_1_id': 'north-carolina-state', 'game_time': '17:05', 'round_num': '5', 'competition_id': '5'}
data=[{'game_id': 259, 'game_date': '2024-03-30', 'team_1_id': 'connecticut', 'team_2_id': 'illinois', 'game_time': '18:09:00', 'round_num': 5, 'competition_id': 5, 'league2_unique': 'ncaambb', 'league1_unique': None}, {'game_id': 260, 'game_date': '2024-03-30', 'team_1_id': 'alabama', 'team_2_id': 'clemson', 'game_time': '20:49:00', 'round_num': 5, 'competition_id': 5, 'league2_unique': 'ncaambb', 'league1_unique': None}, {

In [16]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = COMPETITION_ID, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    print(data)
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"2024/{date}-game-scoring-2024-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [20]:
# date = datetime.date.today()
# date = "2024-03-28"
date = TODAY
date= "2024-03-31"
generate_game_scoring_sheet(competition_id=COMPETITION_ID, date=date)

data=[{'player_unique': 'zach-edey-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competition_id': 5}, {'player_unique': 'braden-smith-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competition_id': 5}, {'player_unique': 'lance-jones-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competition_id': 5}, {'player_unique': 'fletcher-loyer-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competition_id': 5}, {'player_unique': 'mason-gillis-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competition_id': 5}, {'player_unique': 'trey-kaufman-renn-1', 'team_unique': 'purdue', 'game_id': 261, 'game_date': '2024-03-31', 'game_time': '14:20:00', 'round_num': 5, 'competitio

In [56]:
print(TODAY)

2024-03-23


In [10]:
def update_scores_from_csv(current_round: int, competition_id: int, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [12]:
# date = datetime.date.today()
current_round = CURRENT_ROUND
# date = TODAY
date = "2024-03-29"
update_scores_from_csv(current_round=current_round, competition_id=COMPETITION_ID, game_scoring_csv = f"2024/{date}-game-scoring-2024-ncaa-tournament.csv")


losers
data=[{'team_unique': 'marquette', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 2, 'overall_seed': 7, 'region': 'SOUTH', 'round_eliminated': 4, 'round_started': 2, 'team_stats': {'pace': 69.3, 'conference': 'big-east', 'offensive_rating': 112.6, 'assist_percentage': 54.3, 'turnover_percentage': 12.7, 'strength_of_schedule': 10.99, 'free_throw_attempt_rate': 0.256, 'three_point_attempt_rate': 0.405, 'true_shooting_percentage': 0.573, 'effective_field_goal_percentage': 0.551, 'two_point_field_goal_percentage': 0.56, 'three_point_field_goal_percentage': 0.358, 'opp_effective_field_goal_percentage': 0.497}, 'stats_thru': '2024-03-17', 'team_win_loss': {'wins': 25.0, 'losses': 9.0, 'games_played': 0, 'conference_wins': 14.0, 'conference_losses': 6.0}}] count=None
losers
data=[{'team_unique': 'gonzaga', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 5, 'overall_seed': 21, 'region': 'MIDWEST', 'round_eliminated': 4, 'round_started': 2, 'team_stats': {'pace': 70.2,

In [64]:
print(CURRENT_ROUND)

3
